# Understanding Memory


In [4]:
from langchain_openai import ChatOpenAI
from langchain.schema.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from dotenv import load_dotenv
import os

load_dotenv()

def create_basic_chain():
    chatModel = ChatOpenAI(temperature=0.7, verbose=True)
    def processMessage(userQuestion : str):
        sysMessage = SystemMessage(content='''
        You are a helpful assistant! Your name is Bob.
        ''')
    
        humanMessage = HumanMessage(content=userQuestion)
        return chatModel.invoke([sysMessage, humanMessage])
    return processMessage

def main():
    qa_chain = create_basic_chain()
    print("Welcome! I'm Bob, your AI assistant.")
    print("Type 'quit' to exit\n")
    
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'quit':
            break
            
        try:
            response = qa_chain(user_input)
            print("\nBob:", response.content)
        except Exception as e:
            print(f"\nError: {str(e)}")

if __name__ == "__main__":
    if not os.getenv("OPENAI_API_KEY"):
        print("Please set your OPENAI_API_KEY environment variable")
    else:
        main()

Welcome! I'm Bob, your AI assistant.
Type 'quit' to exit


Bob: Hello! How can I assist you today?

Bob: Hello Spock! How can I assist you today?

Bob: Your name is not provided to me. How can I assist you today?

Bob: Your name is not available to me as an AI assistant. How may I assist you further?


## Types of Memory
1. Short term Memory 
2. Long term Memory 

## 2. Adding Short-Term Memory


In [6]:
from langchain.memory import ConversationBufferMemory

def create_chain_with_memory():
    chatModel = ChatOpenAI(temperature=0.7, verbose=True)
    memory = ConversationBufferMemory(
        return_messages=True,  # Returns messages in a structured format
        memory_key="chat_history"  # Where to store the memory
    )
    
    def processMessage(userQuestion: str):
        # Get previous chat history
        chat_history = memory.chat_memory.messages
        
        # Create system message
        sysMessage = SystemMessage(content='''
        You are a helpful assistant! Your name is Bob.
        ''')
        
        # Add new human message
        humanMessage = HumanMessage(content=userQuestion)
        
        # Combine all messages
        messages = [sysMessage] + chat_history + [humanMessage]
        
        # Get response
        response = chatModel.invoke(messages)
        
        # Save the interaction to memory
        memory.chat_memory.add_user_message(userQuestion)
        memory.chat_memory.add_ai_message(response.content)
        
        return response
    
    return processMessage

chain = create_chain_with_memory()

response1 = chain("Hi Bob, my name is Starship commander")
print(response1.content)

response2 = chain("What's my name?")
print(response2.content)

Hello Starship commander! How can I assist you today?
Your name is Starship commander. How can I assist you further?


## 3. Managing Long Conversation History


In [7]:
def create_chain_with_managed_memory():
    chatModel = ChatOpenAI(temperature=0.7, verbose=True)
    
    # Initialize memory with a limit
    memory = ConversationBufferMemory(
        return_messages=True,
        memory_key="chat_history",
        k=5  # Keep only last 5 interactions
    )
    
    def processMessage(userQuestion: str):
        # Get managed chat history
        chat_history = memory.chat_memory.messages[-5:]  # Only last 5 messages
        
        sysMessage = SystemMessage(content='''
        You are a helpful assistant! Your name is Bob. 
        ''')
        
        humanMessage = HumanMessage(content=userQuestion)
        
        # Combine all messages
        messages = [sysMessage] + chat_history + [humanMessage]
        
        # Get response
        response = chatModel.invoke(messages)
        
        # Save to memory
        memory.chat_memory.add_user_message(userQuestion)
        memory.chat_memory.add_ai_message(response.content)
        
        # Log memory state
        print("\nCurrent Memory State:")
        print(f"Number of messages in memory: {len(memory.chat_memory.messages)}")
        
        return response

    return processMessage

def main():
    chain = create_chain_with_managed_memory()
    
    print("Welcome to the Memory-Managed Chat System!")
    print("Type 'quit' to exit\n")
    
    while True:
        user_question = input("\nYou: ")
        
        if user_question.lower() == 'quit':
            break
            
        try:
            response = chain(user_question)
            print("\nBob:", response.content)
            
        except Exception as e:
            print(f"\nAn error occurred: {str(e)}")
            print("Please make sure your OpenAI API key is set correctly.")

if __name__ == "__main__":
    if not os.getenv("OPENAI_API_KEY"):
        print("Please set your OPENAI_API_KEY environment variable")
    else:
        main()

Welcome to the Memory-Managed Chat System!
Type 'quit' to exit


Current Memory State:
Number of messages in memory: 2

Bob: Hello Spock! How can I assist you today?

Current Memory State:
Number of messages in memory: 4

Bob: Pizza is considered delicious by many people because it combines various tasty ingredients like cheese, tomato sauce, and toppings on a warm, crispy crust. The combination of flavors, textures, and aromas in pizza can be very satisfying to the taste buds. Additionally, the customizable nature of pizza allows people to create their own unique combinations to suit their preferences. Overall, the appeal of pizza lies in its versatility and ability to be enjoyed by a wide range of people.

Current Memory State:
Number of messages in memory: 6

Bob: The perimeter of the Earth, also known as the circumference, is approximately 24,901 miles (40,075 kilometers) around the equator. This measurement represents the distance around the Earth at its widest point.

Current Mem

## 4. Using Summary Memory


In [ ]:
from langchain.memory import ConversationSummaryMemory


load_dotenv()

def create_chain_with_summary_memory():
    chatModel = ChatOpenAI(temperature=0.7, verbose=True)
    
    # Initialize summary memory
    memory = ConversationSummaryMemory(
        llm=chatModel,
        return_messages=True,
        memory_key="chat_history"
    )
    
    def processMessage(userQuestion: str):
        # Get the current chat history
        chat_history = memory.chat_memory.messages
        
        sysMessage = SystemMessage(content='''
        You are a helpful assistant! Your name is Bob. 
        ''')
        
        humanMessage = HumanMessage(content=userQuestion)
        messages = [sysMessage] + chat_history + [humanMessage]
        
        # Get response
        response = chatModel.invoke(messages)
        
        # Add to memory
        memory.chat_memory.add_user_message(userQuestion)
        memory.chat_memory.add_ai_message(response.content)
        
        # Print current memory state
        print("\nCurrent Memory State:")
        print(f"Number of messages: {len(memory.chat_memory.messages)}")
        
        # If conversation is long, show the current summary
        if len(memory.chat_memory.messages) > 4:
            print("\nCurrent Conversation Summary:")
            print(memory.predict_new_summary(
                messages=memory.chat_memory.messages,
                existing_summary=""
            ))
        
        return response

    return processMessage

def main():
    chain = create_chain_with_summary_memory()
    
    print("Welcome to the Summary Memory Chat System!")
    print("Type 'quit' to exit\n")
    print("Note: After 4 interactions, you'll see a summary of the conversation\n")
    
    while True:
        user_question = input("\nYou: ")
        
        if user_question.lower() == 'quit':
            break
            
        try:
            response = chain(user_question)
            print("\nBob:", response.content)
            
        except Exception as e:
            print(f"\nAn error occurred: {str(e)}")
            print("Please make sure your OpenAI API key is set correctly.")

if __name__ == "__main__":
    if not os.getenv("OPENAI_API_KEY"):
        print("Please set your OPENAI_API_KEY environment variable")
    else:
        main()

Welcome to the Summary Memory Chat System!
Type 'quit' to exit

Note: After 4 interactions, you'll see a summary of the conversation


Current Memory State:
Number of messages: 2

Bob: Hello, Spock! How can I assist you today?

Current Memory State:
Number of messages: 4

Bob: AI stands for artificial intelligence. It refers to the simulation of human intelligence processes by machines, especially computer systems. AI encompasses tasks such as learning, reasoning, problem-solving, perception, and language understanding. It is a rapidly evolving field with the goal of creating machines that can perform tasks that typically require human intelligence. Let me know if you need more information or have any other questions!

Current Memory State:
Number of messages: 6

Current Conversation Summary:
The human introduces themselves as Spock and asks the AI about artificial intelligence and neural networks. The AI explains that artificial intelligence simulates human intelligence processes and 